In [ ]:
# Set TensorFlow version to 1.x
%tensorflow_version 1.x

# Print assigned GPU
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
    print('and then re-execute this cell.')
else:
    print(gpu_info)

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# IMPORTANT: Ensure that the repository directory follows the path (or update all paths in the script):
# "/content/gdrive/MyDrive/Colab Data/COCO-Human-Pose"

In [ ]:
# Download dataset to pre-specified folder on local VM instance
%cd /content/

import time
from datetime import datetime, timedelta

start = time.time()

!bash "/content/gdrive/MyDrive/Colab Data/COCO-Human-Pose/scripts/coco_dl.sh" ./datasets

download_time = time.time() - start
print("Total download time: {}".format(str(timedelta(seconds=download_time))))

In [ ]:
!ls /content/datasets/coco/train2017 | wc -l
!ls /content/datasets/coco/val2017 | wc -l

In [ ]:
# Ensure that filesystem is set up correctly and will not be the bottleneck
import time
from datetime import datetime, timedelta

start = time.time()

import os
def print_files_in_dir(path):
    print(len(os.listdir(path)), " files at ", path)

print_files_in_dir('/content/datasets/coco/train2017/')
print_files_in_dir('/content/datasets/coco/val2017/')


setup_time = time.time() - start
print("Total setup time: {}".format(str(timedelta(seconds=setup_time))))

if setup_time > 1:
    print("There appears to be a bottleneck with filesystem loading time. This may severely impact training speed.")

In [ ]:
# Change dir to project dir
%cd "/content/gdrive/MyDrive/Colab Data/COCO-Human-Pose"

# Toggle COLAB_TRAINING variable in constants file
# https://askubuntu.com/questions/20414/find-and-replace-text-within-a-file-using-commands
!sed -i.bak 's/COLAB_TRAINING = False/COLAB_TRAINING = True/g' constants.py 

# Remove backup file
!rm constants.py.bak

In [ ]:
!python3 train.py --epochs 100 --batch 12 --hourglass 4 --subset 1.0 --loss "euclidean_loss"

# To resume, uncomment the following and point it to the subdirectory. Ensure hourglass matches.
# !python3 train.py --epochs 100 --batch 12 --hourglass 4 --subset 1.0 --loss "euclidean_loss" --resume True --resume-subdir "2021-03-13-16h-38m_batchsize_12_hg_4" --resume-epoch 9

# Note about subset: if you decide to run on a subset <1.0, there is no functionality to retrieve the same subset of data if the model is stopped then resumed. For this reason, subset should only be used for quick tests rather than reporting robust/repeatable results. 